# Домашнее задание # 1

## Задание 1

In [ ]:
import json
import numpy as np
import random

In [ ]:
np.random.seed(13)
random.seed(13)

In [ ]:
f = open('C:/Users/anastasiia.verbitska/Desktop/Machine Learning/hw02/classification_params_example.json', 'r')
example = json.load(f)
print(example)

Загружаем данные

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target

Разбиваем данные на обучающую и тестовую выборки

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state = 13)

Оцениваем логистическую регрессию

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [ ]:
lr = LogisticRegression(random_state = 13)
cv = cross_val_score(lr, X_train, y_train, cv = 3, scoring = 'accuracy')
print(cv)
print(np.mean(cv))

In [ ]:
lr = LogisticRegression(random_state = 13)
lr.fit(X_train, y_train)

In [ ]:
accuracy_score(lr.predict(X_train), y_train)

In [ ]:
accuracy_score(lr.predict(X_test), y_test)

Находим лучшую модель исходя из заданного набора параметров

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
lr = LogisticRegression(random_state = 13)
params = {'C': np.arange(0.1, 10, 0.1), 'penalty':['l1','l2']}
gscv = GridSearchCV(lr, params, cv = 3)

cv = cross_val_score(gscv, X_train, y_train, cv = 3, scoring = 'accuracy')
print(cv)
print(np.mean(cv))

In [ ]:
lr = LogisticRegression(random_state = 13)
params = {'C': np.arange(0.1, 10, 0.1), 'penalty':['l1','l2']}
gscv = GridSearchCV(lr, params, cv = 3)
gscv.fit(X_train, y_train)

In [ ]:
accuracy_score(gscv.predict(X_train), y_train)

In [ ]:
accuracy_score(gscv.predict(X_test), y_test)

In [ ]:
gscv.best_estimator_.C

In [ ]:
gscv.best_estimator_.penalty

Сохраняем результаты в .json файл

In [ ]:
results = {'C' : gscv.best_estimator_.C, 'penalty'  : gscv.best_estimator_.penalty, 
           'random_state' : gscv.best_estimator_.random_state}

In [ ]:
with open('classification_params_result.json', 'w') as f:
  json.dump(results, f)

## Задание 2

In [ ]:
# %load svm_impl_result.py
import numpy as np
from sklearn.base import BaseEstimator

SVM_PARAMS_DICT = {
    'C': 10.,
    'random_state': 13,
    'iters': 10000,
    'batch_size': 100,
    'step': 0.015
}


class MySVM(BaseEstimator):
    def __init__(self, C, random_state, iters, batch_size, step):
        self.C = C
        self.random_state = random_state
        self.iters = iters
        self.batch_size = batch_size
        self.step = step

    # будем пользоваться этой функцией для подсчёта <w, x>
    def __predict(self, X):
        return np.dot(X, self.w) + self.w0

    # sklearn нужно, чтобы predict возвращал классы, поэтому оборачиваем наш __predict в это
    def predict(self, X):
        res = self.__predict(X)
        res[res > 0] = 1
        res[res < 0] = 0
        return res

    # производная регуляризатора
    def der_reg(self):
        return 1. / self.C * self.w

    # будем считать стохастический градиент не на одном элементе, а сразу на пачке (чтобы было эффективнее)
    def der_loss(self, x, y):
        # s.shape == (batch_size, features)
        # y.shape == (batch_size,)

        # считаем производную по каждой координате на каждом объекте
        # TODO
        M = self.__predict(x) * y.transpose()
        func = lambda x: -1.0 if 1 - x > 0 else 0.0
        vfunc = np.vectorize(func)
        loss = vfunc(M)

        # занулить производные там, где отступ > 1
        # TODO
        loss[M > 1] = 0

        # для масштаба возвращаем средний градиент по пачке
        # TODO
        return np.mean(loss)

    def fit(self, X_train, y_train):
        # RandomState для воспроизводитмости
        random_gen = np.random.RandomState(self.random_state)
        
        # получаем размерности матрицы
        size, dim = X_train.shape
        
        # случайная начальная инициализация
        self.w = random_gen.rand(dim)
        self.w0 = random_gen.randn(1)

        for i in range(self.iters):  
            # берём случайный набор элементов
            rand_indices = random_gen.choice(size, self.batch_size)
            x = X_train[rand_indices]
            y = y_train[rand_indices] * 2 - 1 # исходные метки классов это 0/1 а нам надо -1/1

            # считаем производные
            # TODO
            grad = self.der_loss(x, y)

            # обновляемся по антиградиенту
            # TODO
            self.w = self.w - self.step * (np.dot(y, x) * grad - self.der_reg())
            self.w0 = self.w0 - self.step * np.dot(y, np.ones(100)) * grad

        # метод fit для sklearn должен возвращать self
        return self